In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import PowerTransformer, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

import statsmodels.api as sm
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

RAND_STATE = 32 # for reproducible shuffling
TT_RATIO = 0.25 # test/train

from sklearn.linear_model import Lasso,Ridge,ElasticNet, LinearRegression

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
import warnings
warnings.filterwarnings('ignore')

In [2]:
data_budapest = pd.read_csv('data_budapest.csv')

In [3]:
data_budapest.corr()

,Unnamed: 0,realSum,person_capacity,multi,biz,cleanliness_rating,guest_satisfaction_overall,bedrooms,dist,metro_dist,attr_index_norm,rest_index_norm,room_shared_True,room_private_True,host_is_superhost_True,weekend_yes
Unnamed: 0,1.000000,0.059576,0.041433,-0.039129,0.058745,-0.038094,-0.057458,0.035009,0.120991,0.062892,-0.236188,-0.092657,0.058398,0.031634,-0.072829,-0.054076
realSum,0.059576,1.000000,0.235012,-0.087597,0.040852,0.025161,-0.001317,0.211160,-0.092137,-0.082865,0.123350,0.102905,-0.022393,-0.175221,-0.043701,0.063614
person_capacity,0.041433,0.235012,1.000000,-0.082114,0.050778,-0.004296,0.001638,0.532221,-0.082495,-0.062539,0.069816,0.069129,0.001442,-0.285407,-0.046154,-0.031453
multi,-0.039129,-0.087597,-0.082114,1.000000,-0.482955,0.086402,0.084712,-0.004823,0.065497,0.073906,-0.035514,-0.004609,-0.029815,0.141468,0.078849,0.010941
biz,0.058745,0.040852,0.050778,-0.482955,1.000000,-0.128630,-0.150562,-0.064682,-0.175337,-0.157458,0.151751,0.186232,0.045319,-0.034427,-0.067339,-0.009941
cleanliness_rating,-0.038094,0.025161,-0.004296,0.086402,-0.128630,1.000000,0.703926,0.018288,-0.011083,0.001526,0.004096,0.030799,0.001587,-0.078259,0.276447,0.010082
guest_satisfaction_overall,-0.057458,-0.001317,0.001638,0.084712,-0.150562,0.703926,1.000000,0.020004,-0.011930,0.008564,0.010689,0.024061,0.027048,-0.073748,0.290947,0.009216
bedrooms,0.035009,0.211160,0.532221,-0.004823,-0.064682,0.018288,0.020004,1.000000,0.053162,0.048873,0.003667,-0.037064,-0.009414,-0.034689,-0.047929,-0.026130
dist,0.120991,-0.092137,-0.082495,0.065497,-0.175337,-0.011083,-0.011930,0.053162,1.000000,0.830051,-0.603398,-0.679947,0.077672,0.242451,-0.070597,-0.014880
metro_dist,0.062892,-0.082865,-0.062539,0.073906,-0.157458,0.001526,0.008564,0.048873,0.830051,1.000000,-0.370594,-0.410524,0.040400,0.220510,-0.058303,-0.014941


In [4]:
data_budapest = data_budapest.drop(['Unnamed: 0'], axis=1)

In [5]:
x = data_budapest.drop('realSum', axis=1)
y = data_budapest.realSum

In [6]:
# test-train split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=TT_RATIO, random_state=RAND_STATE)
x_train = pd.DataFrame(x_train)
x_test = pd.DataFrame(x_test)
x_train.describe()

,person_capacity,multi,biz,cleanliness_rating,guest_satisfaction_overall,bedrooms,dist,metro_dist,attr_index_norm,rest_index_norm,room_shared_True,room_private_True,host_is_superhost_True,weekend_yes
count,3016.000000,3016.000000,3016.000000,3016.000000,3016.000000,3016.000000,3016.000000,3016.000000,3016.000000,3016.000000,3016.000000,3016.000000,3016.000000,3016.000000
mean,3.542772,0.303714,0.348475,9.491048,94.681366,1.111737,1.886485,0.545259,12.629868,34.448628,0.003979,0.100796,0.375995,0.481764
std,1.260509,0.459937,0.476566,0.821308,6.360919,0.671461,1.914215,0.886999,6.748639,19.082618,0.062962,0.301108,0.484459,0.499750
min,2.000000,0.000000,0.000000,2.000000,20.000000,0.000000,0.060000,0.020000,0.926301,1.894796,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,0.000000,9.000000,93.000000,1.000000,0.880000,0.230000,8.070396,19.637764,0.000000,0.000000,0.000000,0.000000
50%,4.000000,0.000000,0.000000,10.000000,96.000000,1.000000,1.490000,0.340000,11.860541,29.355339,0.000000,0.000000,0.000000,0.000000
75%,4.000000,1.000000,1.000000,10.000000,99.000000,1.000000,2.210000,0.520000,15.793290,50.340155,0.000000,0.000000,1.000000,1.000000
max,6.000000,1.000000,1.000000,10.000000,100.000000,3.000000,19.250000,11.690000,100.000000,100.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
x_train_const = sm.add_constant(x_train) # adding a constant

model = sm.OLS(y_train, x_train_const).fit()
predictions_train = model.predict(x_train_const)

x_test_const = sm.add_constant(x_test) # adding a constant
predictions_test = model.predict(x_test_const)
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                realSum   R-squared:                       0.096
Model:                            OLS   Adj. R-squared:                  0.091
Method:                 Least Squares   F-statistic:                     22.64
Date:                Mon, 06 Mar 2023   Prob (F-statistic):           4.78e-56
Time:                        17:03:21   Log-Likelihood:                -19062.
No. Observations:                3016   AIC:                         3.815e+04
Df Residuals:                    3001   BIC:                         3.824e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
const               

In [8]:
sts = StandardScaler()

x_train_standarized = sts.fit_transform(x_train)
x_test_standarized = sts.transform(x_test)

x_train_standarized = pd.DataFrame(x_train_standarized)

In [9]:
x_train_const = sm.add_constant(x_train_standarized.to_numpy()) # adding a constant

model = sm.OLS(y_train, x_train_const).fit()
predictions_train = model.predict(x_train_const)

x_test_const = sm.add_constant(x_test_standarized) # adding a constant
predictions_test = model.predict(x_test_const)
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                realSum   R-squared:                       0.096
Model:                            OLS   Adj. R-squared:                  0.091
Method:                 Least Squares   F-statistic:                     22.64
Date:                Mon, 06 Mar 2023   Prob (F-statistic):           4.78e-56
Time:                        17:03:21   Log-Likelihood:                -19062.
No. Observations:                3016   AIC:                         3.815e+04
Df Residuals:                    3001   BIC:                         3.824e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        178.4292      2.454     72.698      0.0

In [10]:
lm = LinearRegression()
model = lm.fit(x_train,y_train)
model.coef_
model.intercept_

38.18982661663483

In [11]:
model.score(x_train, y_train)

0.09552019770431008

In [12]:
y_pred = pd.DataFrame(model.predict(x_test),columns = ['realSum'] )      # model prediction
y_pred_train =  pd.DataFrame(model.predict(x_train),columns = ['realSum'])

In [13]:
mse(y_test,y_pred)

7407.924644344295

In [14]:
rmse = np.sqrt(mse(y_test,y_pred))
rmse

86.06930140499745

In [15]:
##prediction on the train set
mse(y_train,y_pred_train)

18078.178193151438

In [16]:
mae(y_test,y_pred)

50.01311948849357

In [17]:
R2=r2_score(y_test,y_pred)
R2

0.15610367406769043

In [18]:
R2_test=model.score(x_test,y_test)
R2_train=model.score(x_train,y_train)
Adj_R2= 1 - (1-R2)*(len(y_test)-1)/(len(y_test)-x_test.shape[1]-1)
Adj_R2

0.14418182889811193

In [19]:
model_lasso=Lasso(alpha=0.05)

model_lasso.fit(x_train, y_train)
print(f"{model_lasso.__class__.__name__}: Train -> {model_lasso.score(x_train, y_train)}, Test -> {model_lasso.score(x_test, y_test)}")

Lasso: Train -> 0.09548028936254316, Test -> 0.15686897417127366


In [20]:
model_ridge=Ridge(alpha=1000)
model_ridge.fit(x_train, y_train)
print(f"{model_ridge.__class__.__name__}: Train -> {model_ridge.score(x_train, y_train)}, Test -> {model_ridge.score(x_test, y_test)}")

Ridge: Train -> 0.07968575981487203, Test -> 0.15524250139127993


In [21]:
model = DecisionTreeRegressor()
model.fit(x_train, y_train)

print("X_train has %d rows and %d columns"  %(x_train.shape[0],x_train.shape[1]))
print("-----------------------------------")
print("The coefficient of determination for the test data is R2=%.2f"
      %(model.score(x_test, y_test)))
print("The coefficient of determination for the train data is R2=%.2f"
      %(model.score(x_train, y_train)))


X_train has 3016 rows and 14 columns
-----------------------------------
The coefficient of determination for the test data is R2=0.05
The coefficient of determination for the train data is R2=1.00


In [22]:
scores=cross_val_score(model, x_train, y_train, cv=5)
print("Cross validation scores: ", scores)
print("Score stats: %0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

Cross validation scores:  [-0.09400352  0.02141055 -0.11192253  0.65608053 -0.02810516]
Score stats: 0.09 accuracy with a standard deviation of 0.29


In [23]:
model1 = DecisionTreeRegressor()
model2 = LinearRegression()
model3 = KNeighborsRegressor()

model_pipeline = [model1, model2, model3]
model_names = ['Regression Tree', 'Linear Regression', 'KNN']
scores = {}
i=0
for model in model_pipeline:
    mean_score = np.mean(cross_val_score(model, x_train, y_train, cv=5))
    scores[model_names[i]] = mean_score
    i = i+1
print(scores)

{'Regression Tree': -0.06512491275104133, 'Linear Regression': 0.11436728247774941, 'KNN': -0.20538304858467}
